In [1]:
import os
import requests
from dotenv import load_dotenv

from langchain_openai import AzureChatOpenAI
from openai import AzureOpenAI

from langchain.chat_models import init_chat_model

from langchain.tools import tool
from langchain.agents import create_agent

pwd = %pwd
env_path = os.path.join(pwd,'.env')

load_dotenv(env_path)


/mnt/custom-file-systems/efs/fs-0713103aada1ad94d_fsap-03d52ef0d5b91b6d3/LLM-World/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [7]:
os.getenv('AZURE_OPENAI_BASEURL')

'https://bms-openai-proxy-eus-prod.azu.bms.com/openai-urls.json'

In [12]:

response = requests.get(os.getenv('AZURE_OPENAI_BASEURL'))
response.json()['nonprod'].keys()
response.json()['nonprod']['dall-e-3']


[{'azure_region': 'australiaeast',
  'deployment_name': 'dall-e-3',
  'deployment_type': 'Standard',
  'endpoint': 'https://bms-openai-services-australiaeast-1-nonprod.azu.bms.com',
  'model_version': '3.0',
  'requests_per_minute_across_bms': 12,
  'tokens_per_minute_across_bms': 2000},
 {'azure_region': 'eastus',
  'deployment_name': 'dall-e-3',
  'deployment_type': 'Standard',
  'endpoint': 'https://bms-openai-services-eastus-1-nonprod.azu.bms.com',
  'model_version': '3.0',
  'requests_per_minute_across_bms': 12,
  'tokens_per_minute_across_bms': 2000},
 {'azure_region': 'swedencentral',
  'deployment_name': 'dall-e-3',
  'deployment_type': 'Standard',
  'endpoint': 'https://bms-openai-services-swedencentral-1-nonprod.azu.bms.com',
  'model_version': '3.0',
  'requests_per_minute_across_bms': 12,
  'tokens_per_minute_across_bms': 2000}]

In [20]:
model_endpoint = {}
for obj_name in response.json()['nonprod']:
    model_endpoint[response.json()['nonprod'][obj_name][0]['deployment_name']]=response.json()['nonprod'][obj_name][0]['endpoint']
    # print(response.json()['nonprod'][obj_name][0])
    # print(response.json()['nonprod'][obj_name][0]['deployment_name'])
    # print(response.json()['nonprod'][obj_name][0]['endpoint'])
model_endpoint.keys()

dict_keys(['dall-e-3', 'gpt-4.1', 'gpt-4.1-mini', 'gpt-4.1-nano', 'gpt-4o-audio-preview', 'gpt-4o-global', 'gpt-4o-mini-global', 'gpt-4o-regional', 'gpt-5', 'gpt-5-chat', 'gpt-5-mini', 'gpt-5-nano', 'gpt-5.1', 'gpt-5.1-chat', 'gpt-audio', 'gpt-audio-mini', 'gpt-image-1', 'gpt-image-1-mini', 'model-router', 'o1', 'o3', 'o3-mini', 'o4-mini', 'text-embedding-3-large', 'text-embedding-3-small', 'text-embedding-ada-002'])

In [7]:
def openai_llm(modle_id: str):
    '''Create LangChain LLM object for Azure OpenAI llm'''
    api_response = requests.get(os.getenv('AZURE_OPENAI_BASEURL'))
    payload = api_response.json()

    gllm = AzureChatOpenAI(
        deployment_name=modle_id,  
        openai_api_version="2024-12-01-preview",
        azure_endpoint=payload['nonprod'][modle_id][0]['endpoint'],
        api_key=os.getenv('AZURE_OPENAI_KEY'),
        temperature=0)
    return gllm


In [ ]:
os.environ['AZURE_OPENAI_API_VERSION']

In [ ]:
model

In [ ]:
@tool
def create_calendar_event(
    title:str,
    start_time:str,
    end_time:str,
    atendees:list[str],
    location:str=""
) -> str:
    """create calendar event"""
    return f"Created event: {title} from {start_time} to {end_time} with {len(atendees)} atendees."

@tool
def send_email(
    to:list[str],
    subject:str,
    body:str,
    cc:list[str]
) -> str:
    """send an email via api"""
    return f"Email sent to {','.join(to)} - Subject: {subject}"


@tool
def get_available_time_slots(
    atendees:list[str],
    date:str,
    duration_minutes:int,
)-> list[str]:
    """Check calendar availability for a given atendee on a specific date"""
    return ["09:00","14:00","16:00"]

In [ ]:
CALENDAR_AGENT_PROMPT = (
    "You are a calendar scheduling assistant. "
    "Parse natural language scheduling requests (eg, 'next Tuesday at 2pm') "
    "into proper ISO datetime formats. "
    "Use get_available_time_slots to check availability when needed. "
    "Use create_calendar_event to schedule events. "
    "Always confirm what was scheduled in your final response. "
)



calendar_agent = create_agent(
    model,
    tools=[create_calendar_event, get_available_time_slots],
    system_prompt=CALENDAR_AGENT_PROMPT
)

In [ ]:
calendar_agent

In [ ]:
## test agent

query = "Schedule a team meeting next Tuesday at 2pm for 1 hour"

## surface real-time response from agent
for step in calendar_agent.stream(
    {"messages":[{"role":"user","content":query}]}, 
    #stream_mode="updates" #emits an event after every step
):
    print(step.values())